<a href="https://colab.research.google.com/github/feliciamargareta/Preparing-Data-for-Customer-Analytics-Modeling/blob/main/Preparing_Data_for_Customer_Analytics_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called Training Data Ltd. to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to customer_train.csv, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

- student_id:	A unique ID for each student.
- city:	A code for the city the student lives in.
- city_development_index:	A scaled development index for the city.
- gender:	The student's gender.
- relevant_experience:	An indicator of the student's work relevant experience.
- enrolled_university:	The type of university course enrolled in (if any).
- education_level:	The student's education level.
- major_discipline:	The educational discipline of the student.
- experience:	The student's total work experience (in years).
- company_size:	The number of employees at the student's current employer.
- last_new_job:	The number of years between the student's current and previous jobs.
- training_hours:	The number of hours of training completed.
- job_change:	An indicator of whether the student is looking for a new job (1) or not (0).

## Exploratory data analysis
Load customer_train.csv to begin exploring the data to understand the contents and data types of the values in each column.

In [1]:
# import packages
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# Copy the DataFrame for cleaning
ds_jobs_clean = ds_jobs.copy()
ds_jobs_clean.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0


In [2]:
# check the data type of each column
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

## Converting data types

In [3]:
# check the ordinal column unique content
print(ds_jobs_clean['relevant_experience'].unique())
print(ds_jobs_clean['enrolled_university'].unique())
print(ds_jobs_clean['education_level'].unique())
print(ds_jobs_clean['experience'].unique())
print(ds_jobs_clean['company_size'].unique())
print(ds_jobs_clean['last_new_job'].unique())

['Has relevant experience' 'No relevant experience']
['no_enrollment' 'Full time course' nan 'Part time course']
['Graduate' 'Masters' 'High School' nan 'Phd' 'Primary School']
['>20' '15' '5' '<1' '11' '13' '7' '17' '2' '16' '1' '4' '10' '14' '18'
 '19' '12' '3' '6' '9' '8' '20' nan]
[nan '50-99' '<10' '10000+' '5000-9999' '1000-4999' '10-49' '100-499'
 '500-999']
['1' '>4' 'never' '4' '3' '2' nan]


## Converting ordered categories
Convert columns containing ordinal categorical data into ordered categories.

In [4]:
# Create a dictionary of columns containing ordered categorical data
ordered_cats = {
    'relevant_experience': ['No relevant experience', 'Has relevant experience'],
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

In [5]:
# Loop through DataFrame columns to efficiently change data types
for col in ds_jobs_clean:

    # Convert integer columns to int32
    if ds_jobs_clean[col].dtype == 'int':
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('int32')

    # Convert float columns to float16
    elif ds_jobs_clean[col].dtype == 'float':
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('float16')

    # Convert columns containing ordered categorical data to ordered categories using dict
    elif col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
        ds_jobs_clean[col] = ds_jobs_clean[col].astype(category)

    # Convert remaining columns to standard categories
    else:
        ds_jobs_clean[col] = ds_jobs_clean[col].astype('category')


In [7]:
# re-check the data type after converting
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   int32   
dtypes: category(10), float1

We can see that all the data type has been adjusted to expected data type.
We can also see that the memory usage also dropped significantly from 2.0+ MB to only 76.1 KB

## Reordering columns
Reorder the columns of ds_jobs_clean so it has the same column ordering as the original dataset.

In [6]:
# Filter students with 10 or more years experience at companies with at least 1000 employees
ds_jobs_clean = ds_jobs_clean[(ds_jobs_clean['experience'] >= '10') & (ds_jobs_clean['company_size'] >= '1000-4999')]

In [12]:
# check the company size filter
ds_jobs_clean.groupby(['company_size']).size()

company_size
<10             0
10-49           0
50-99           0
100-499         0
500-999         0
1000-4999     796
5000-9999     310
10000+       1095
dtype: int64

In [13]:
# check the experience filter
ds_jobs_clean.groupby(['experience']).size()

experience
<1       0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10     245
11     149
12     107
13     103
14     147
15     172
16     128
17      91
18      68
19      83
20      49
>20    859
dtype: int64

The filter has been applied correctly to the final dataframe